#### 0. Import basic methods (a.o. from source2.py)

In [4]:
from source2 import *
import numpy as np

In [5]:
# load basic info from the Input sheet
mapping = load_input_from_xls(filename=r'Case 1 - Input Sheet.xlsm')
Inputs = extract_scenario(mapping, 'Scenario1')

##### 1. prepare input data and generate objects
We are testing the following cases:
##### 1.1 Turbines

In [6]:
turbine_data = get_object_data(Inputs=Inputs, 
                               subsystem='Wind energy source & Transport', 
                               element='Offshore wind park',
                               component='Turbine')
turbine_data['depreciation_rate']=1/turbine_data['economic_lifetime']
display(turbine_data)

{'sub_system': 'Wind energy source & Transport',
 'element': 'Offshore wind park',
 'component': 'Turbine',
 'escalation_base_year': 2023,
 'escalation_rate': 0.02,
 'capex_per_unit': 1139979.0,
 'capex_per_unit_units': 'EUR / MW',
 'unit': 3000.0,
 'unit_units': 'MW',
 'construction_duration': 3,
 'share_of_investments': [0.2, 0.4, 0.4],
 'economic_lifetime': 30,
 'depreciation_flag': 1,
 'depreciation_rate': 0.03333333333333333,
 'yearly_variable_costs_flag': 1,
 'yearly_variable_costs_rate': 0.03,
 'insurance_flag': 1,
 'insurance_rate': 0.005,
 'decommissioning_rate': 0.08,
 'residual_value': 0.01}

In [7]:
Turbine = Object(**turbine_data)
# hier heb je alleen het object met inputs

In [8]:
Turbine.generate_cashflows(startyear=2023, lifecycle=29, debug=False)
# na deze regel heeft Turbine.df (dataframe met alleen Capex, Opex en Residuals)

##### 1.2 Foundations and cables

In [9]:
foundation_data = get_object_data(Inputs=Inputs, 
                               subsystem='Wind energy source & Transport', 
                               element='Offshore wind park',
                               component='Foundation & cable')
foundation_data['depreciation_rate']=1/foundation_data['economic_lifetime']
display(foundation_data)

{'sub_system': 'Wind energy source & Transport',
 'element': 'Offshore wind park',
 'component': 'Foundation & cable',
 'escalation_base_year': 2023,
 'escalation_rate': 0.02,
 'capex_per_unit': 971621.0,
 'capex_per_unit_units': 'EUR / MW',
 'unit': 3000.0,
 'unit_units': 'MW',
 'construction_duration': 3,
 'share_of_investments': [0.2, 0.4, 0.4],
 'economic_lifetime': 30,
 'depreciation_flag': 1,
 'depreciation_rate': 0.03333333333333333,
 'yearly_variable_costs_flag': 1,
 'yearly_variable_costs_rate': 0.015,
 'insurance_flag': 1,
 'insurance_rate': 0.005,
 'decommissioning_rate': 0.35,
 'residual_value': 0.01}

In [10]:
Foundation = Object(**foundation_data)
# hier heb je alleen het object met inputs

In [11]:
Foundation.generate_cashflows(startyear=2023, lifecycle=29, debug=False)
# na deze regel heeft Turbine.df (dataframe met alleen Capex, Opex en Residuals)

In [12]:
(Turbine.df+Foundation.df).tail()

,years,capex,opex,revenue
years,,,,
2047,4094,0.000000e+00,-3.050565e+08,0
2048,4096,0.000000e+00,-3.111576e+08,0
2049,4098,0.000000e+00,-3.173807e+08,0
2050,4100,0.000000e+00,-3.237283e+08,0
2051,4102,8.823474e+08,-2.730357e+09,0


#####  2. Test values - Turbines
##### 2.1 - CAPEX values

In [13]:
MTBS_values = {'years': [2023, 2024, 2025], 
               'values': [-914_940_000, -1_866_477_600, -1_903_807_151]}

In [14]:
for index, year in enumerate(MTBS_values['years']):
    
    print('CAPEX value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Turbine.df[Turbine.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Turbine.df[Turbine.df.index==year].capex.item(), 0)

CAPEX value testing in 2023: MTBS: -914940000 TUD/VO: -697667148.0


AssertionError: 
Arrays are not almost equal to 0 decimals
 ACTUAL: -914940000
 DESIRED: -697667148.0

##### 2.2 - OPEX values

In [ ]:
MTBS_values = {'years': [2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052],
               'values': [-177_500_328, -181_050_335, -184_671_341, -188_364_768, -192_132_064, -195_974_705, -199_894_199, -203_892_083, -207_969_925, -212_129_323, -216_371_910, -220_699_348, -225_113_335, -229_615_601, -234_207_913, -238_892_072, -243_669_913, -248_543_311, -253_514_178, -258_584_461, -263_756_150, -269_031_273, -274_411_899, -279_900_137, -285_498_140, -291_208_102, -297_032_265]}

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Opex value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Turbine.df[Turbine.df.index==year].opex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Turbine.df[Turbine.df.index==year].opex.item(), 0)

##### 2.3 - Decommissioning + divestment


In [ ]:
MTBS_values = {'years': [2052],
               'values': [-169732722.573259+624696633.5999999]} # decommissioning from MTBS, residual from python

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Decommissioning value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Turbine.df[Turbine.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Turbine.df[Turbine.df.index==year].capex.item(), 0)

#####  3. Test values - Foundations and cables
##### 3.1 - CAPEX values

In [ ]:
MTBS_values = {'years': [2023, 2024, 2025], 
               'values': [-571_732_848, -1_166_335_010, -1_189_661_710]}

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('CAPEX value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Foundation.df[Foundation.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Foundation.df[Foundation.df.index==year].capex.item(), 0)

##### 3.2 - OPEX values

In [ ]:
MTBS_values = {'years': [2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035, 2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046, 2047, 2048, 2049, 2050, 2051, 2052],
               'values': [-63_381_373, -64_649_000, -65_941_980, -67_260_820, -68_606_036, -69_978_157, -71_377_720, -72_805_275, -74_261_380, -75_746_608, -77_261_540, -78_806_771, -80_382_906, -81_990_564, -83_630_375, -85_302_983, -87_009_043, -88_749_223, -90_524_208, -92_334_692, -94_181_386, -96_065_014, -97_986_314, -99_946_040, -101_944_961, -103_983_860, -106_063_537]}

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Opex value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Foundation.df[Foundation.df.index==year].opex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Foundation.df[Foundation.df.index==year].opex.item(), 0)

##### 3.3 - Decommissioning + divestment


In [ ]:
MTBS_values = {'years': [2052],
               'values': [-305518900.631867+1124453940.4800005]} # decommissioning from MTBS, residual from python

In [ ]:
for index, year in enumerate(MTBS_values['years']):
    
    print('Residual value testing in {}: MTBS: {} TUD/VO: {}'.format(year, MTBS_values['values'][index], Foundation.df[Foundation.df.index==year].capex.item()))
    np.testing.assert_almost_equal(MTBS_values['values'][index],  Foundation.df[Foundation.df.index==year].capex.item(), 0)